In [1]:
from neo4j import GraphDatabase
import pandas as pd

### Streamer nodes

In [6]:
pd.read_csv('https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/Streamer_dataset.csv')

,ID_streamer,streamer,minutes_live,viewer_mean,viewer_peak,spect_mean,spect_tot
0,0,LyonWGFLive,1155,14011,19727,9337,58801
1,1,Tumblurr,2925,13683,32436,10655,148019
2,2,ChristianVieriOfficial,570,11014,24931,7056,64614
3,3,GrenBaud,1005,10285,30017,7600,78521
4,4,ZanoXVII,3480,7681,13973,5699,157718
...,...,...,...,...,...,...,...
2972,2972,QLASH_Simracing,180,10,15,5,20
2973,2973,teoKrazia,120,10,15,8,21
2974,2974,yume940,225,10,23,6,27
2975,2975,zanella_productions,165,10,12,8,17


In [7]:
pd.read_csv('https://raw.githubusercontent.com/gianscuri/Twitch_Community_Graph/main/DataProcessing/Games_dataset.csv')

,ID_game,game_name
0,0,NaN
1,1,Animal Crossing: New Horizons
2,2,Astroneer
3,3,Baba is You
4,4,Bloodstained: Ritual of the Night
...,...,...
1140,1140,Poppy Playtime
1141,1141,Final Fantasy IX
1142,1142,Ghost Exile
1143,1143,Higurashi: When They Cry


In [6]:
streamer_constraint = '''
CREATE CONSTRAINT StreamerNameKey IF NOT EXISTS
FOR (s:Streamer)
REQUIRE s.Name IS UNIQUE
'''

In [12]:
# Creo i nodi e per ogni proprietà che aggiungo vado a controllare se nel file il dato è presente: se si, aggiungo la proprietà e inserisco il valore mentre se il dato 
# non è presente non inserisco la proprietà.
# Per la chiave del nodo (in questo caso il nome del pokemon) non faccio il controllo in quanto su neo4j ho inserito il vincolo di KEY che implica che la proprietà esista 
# e che sia unica 
pok_import_csv = "LOAD CSV WITH HEADERS FROM 'https://github.com/gianscuri/Twitch_Community_Graph/blob/main/DataProcessing/Streamer_dataset.csv' AS row"
pok_key = "MERGE (s:Streamer {Name: row.Name})"

'''
pok_nokey = pok.drop('Name', axis=1)
pok_nokey=pok_nokey.columns
pok_nokey = pok_nokey.values.tolist()
pok_execution_commands = []
for i in pok_nokey:
    neo4j_create_statemenet ="FOREACH(ignoreMe IN CASE WHEN trim(row."+ str(i)  + ")" + " <> \"\" THEN [1] ELSE [] END | SET p."+str(i)+" = row."+str(i) +")"
    pok_execution_commands.append(neo4j_create_statemenet)
pok_creation_node = '\n'.join(pok_execution_commands)
pok_final_query = str(pok_import_csv)+'\n'+str(pok_key)+'\n'+str(pok_creation_node)+'\n'+'RETURN p'
print(pok_final_query) # bisogna sistemare i nomi delle proprietà che deevono essere conformi alle convenzioni neo4j'''

'\npok_nokey = pok.drop(\'Name\', axis=1)\npok_nokey=pok_nokey.columns\npok_nokey = pok_nokey.values.tolist()\npok_execution_commands = []\nfor i in pok_nokey:\n    neo4j_create_statemenet ="FOREACH(ignoreMe IN CASE WHEN trim(row."+ str(i)  + ")" + " <> "" THEN [1] ELSE [] END | SET p."+str(i)+" = row."+str(i) +")"\n    pok_execution_commands.append(neo4j_create_statemenet)\npok_creation_node = \'\n\'.join(pok_execution_commands)\npok_final_query = str(pok_import_csv)+\'\n\'+str(pok_key)+\'\n\'+str(pok_creation_node)+\'\n\'+\'RETURN p\'\nprint(pok_final_query) # bisogna sistemare i nomi delle proprietà che deevono essere conformi alle convenzioni neo4j'

In [ ]:
pok_import_nodes = '''
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_pokemon_import.csv' AS row
MERGE (p:Pokemon {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.NationalPokedexNumber) <> "" THEN [1] ELSE [] END | SET p.NationalPokedexNumber = toInteger(row.NationalPokedexNumber))
FOREACH(ignoreMe IN CASE WHEN trim(row.Species) <> "" THEN [1] ELSE [] END | SET p.Species = row.Species)
FOREACH(ignoreMe IN CASE WHEN trim(row.Variant) <> "" THEN [1] ELSE [] END | SET p.Variant = row.Variant)
FOREACH(ignoreMe IN CASE WHEN trim(row.Generation) <> "" THEN [1] ELSE [] END | SET p.Generation = toInteger(row.Generation))
FOREACH(ignoreMe IN CASE WHEN trim(row.Rarity) <> "" THEN [1] ELSE [] END | SET p.Rarity = row.Rarity)
FOREACH(ignoreMe IN CASE WHEN trim(row.Type1) <> "" THEN [1] ELSE [] END | SET p.Type1 = row.Type1)
FOREACH(ignoreMe IN CASE WHEN trim(row.Type2) <> "" THEN [1] ELSE [] END | SET p.Type2 = row.Type2)
FOREACH(ignoreMe IN CASE WHEN trim(row.Total) <> "" THEN [1] ELSE [] END | SET p.Total = toInteger(row.Total))
FOREACH(ignoreMe IN CASE WHEN trim(row.HP) <> "" THEN [1] ELSE [] END | SET p.HP = toInteger(row.HP))
FOREACH(ignoreMe IN CASE WHEN trim(row.Attack) <> "" THEN [1] ELSE [] END | SET p.Attack = toInteger(row.Attack))
FOREACH(ignoreMe IN CASE WHEN trim(row.Defense) <> "" THEN [1] ELSE [] END | SET p.Defense = toInteger(row.Defense))
FOREACH(ignoreMe IN CASE WHEN trim(row.SpAtk) <> "" THEN [1] ELSE [] END | SET p.SpAtk = toInteger(row.SpAtk))
FOREACH(ignoreMe IN CASE WHEN trim(row.SpDef) <> "" THEN [1] ELSE [] END | SET p.SpDef = toInteger(row.SpDef))
FOREACH(ignoreMe IN CASE WHEN trim(row.Speed) <> "" THEN [1] ELSE [] END | SET p.Speed = toInteger(row.Speed))
FOREACH(ignoreMe IN CASE WHEN trim(row.image_url) <> "" THEN [1] ELSE [] END | SET p.image_url = row.image_url)
FOREACH(ignoreMe IN CASE WHEN trim(row.Vgc2022Rules) <> "" THEN [1] ELSE [] END | SET p.Vgc2022Rules = row.Vgc2022Rules)
FOREACH(ignoreMe IN CASE WHEN trim(row.MonthlyUsageMarch2022) <> "" THEN [1] ELSE [] END | SET p.MonthlyUsageMarch2022 = toInteger(row.MonthlyUsageMarch2022))
FOREACH(ignoreMe IN CASE WHEN trim(row.UsagePercentMarch2022) <> "" THEN [1] ELSE [] END | SET p.UsagePercentMarch2022 = toInteger(row.UsagePercentMarch2022))
FOREACH(ignoreMe IN CASE WHEN trim(row.MonthlyRankMarch2022) <> "" THEN [1] ELSE [] END | SET p.MonthlyRankMarch2022 = toInteger(row.MonthlyRankMarch2022))
RETURN p
'''

## Import su Neo4J

In [7]:
data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "twitch"))
session = data_base_connection.session()

In [8]:
# POKEMON: Import vincoli e nodi

session.run(streamer_constraint)
# session.run(pok_import_nodes)

# MOVES: Import vincoli e nodi
# session.run(moves_constraint)
# session.run(moves_import_nodes)

# TYPES: Import vincoli e nodi
# session.run(types_constraint)
# session.run(types_import_nodes)

# ITEMS: Import vincoli e nodi
# session.run(items_constraint)
# session.run(items_import_nodes)

# ABILITIES: Import vincoli e nodi
# session.run(abilities_constraint)
# session.run(abilities_import_nodes)

# NATURES: Import vincoli e nodi
# session.run(natures_constraint)
# session.run(natures_import_nodes)